In [1]:
import pandas as pd
import numpy as np
import psycopg2 as ps
from psycopg2 import sql
from sqlalchemy import create_engine, text
import os
import openpyxl
from datetime import date
import glob

# Set display option to show all columns 
pd.set_option('display.max_columns', None)

# # Change the current working directory
# os.chdir('C:\\Users\\cantr\\OneDrive\\Coding\\GiordanoDB\\Peachtree\\registration_data')

In [77]:
current_directory = os.getcwd()
print(current_directory)

c:\Users\cantr\OneDrive\Coding\GiordanoDB\Peachtree


In [35]:
conn = ps.connect(database="1264bra", user="postgres", password="password", host="localhost", port="5432") 

cur = conn.cursor()

sql_query_leagues = """
    select peachtree_id
            , concat(first_name, ' ', last_name) as player_name
            , parent_name
            , guardian_email
			, league
            , division
            , season_team_name
			, birthdate
			, previous_seasons
			, full_address
			, school_name
			, season
    from peachtree.vw_player_registration
    --where division <> 'Refunded'
"""

cur.execute(sql_query_leagues)

results = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
df_peachtree_history_orig = pd.DataFrame(results, columns=column_names)

###

In [36]:
df_peachtree_history_orig

,peachtree_id,player_name,parent_name,guardian_email,league,division,season_team_name,birthdate,previous_seasons,full_address,school_name,season
0,15,Ainsley Dority,Mandy Dority,mandydority@icloud.com,Refunded,Refunded,8U Penguins,2016-10-07,3,"PO BOX 411, Batesville, VA 22924",Brownsville Elementary,fall 2024
1,11,Addison Pietro,Justin Pietro,jpietro@gmail.com,Refunded,Refunded,8U Penguins,2016-09-08,0,"2939 Rambling Brook Lane, Crozet, VA 22932",Brownsville Elementary,fall 2024
2,734,William Obrien,John OBrien,jmobrien88@gmail.com,Refunded,Refunded,CP Yankees,2018-05-01,0,"1402 Stillhouse Ridge Ln, Charlottesville, VA ...",Brownsville Elementary,fall 2024
3,288,Genevieve Fosdick,Jessica Fosdick,jefosdick@gmail.com,Refunded,Refunded,Tball Athletics,2020-09-04,0,"1554 Wickham Pond Drive, Charlottesville, VA 2...",Bright Beginnings,fall 2024
4,365,James Buckett,Jennifer Buckett,jenbuckett@gmail.com,Baseball,Rookie Baseball,Rookie Yankees,2017-08-30,3,"1042 Rolling Meadow Ln, Crozet, VA 22932",Crozet Elementary,fall 2024
...,...,...,...,...,...,...,...,...,...,...,...,...
1679,591,Parker Lambert,Travis Lambert,lambertw1221@gmail.com,Baseball,Majors Baseball,None,2014-09-03,6,"645 Jonna St, Crozet, VA 22932",Crozet Elementary,spring 2026
1680,556,Natalie Lambert,Travis Lambert,lambertw1221@gmail.com,Softball,8u Softball,None,2017-08-31,4,"645 Jonna St, Crozet, VA 22932",Crozet Elementary,spring 2026
1681,657,Silas Efaw,Robin Shirley,rnshirley@gmail.com,Baseball,Rookie Baseball,None,2019-07-27,4,"803 Arboleda Drive, Crozet, VA 22932",Compass Acadamy Cooperative,spring 2026
1682,369,James O'Neil,Kate ONeil,katecm9@gmail.com,Baseball,Majors Baseball,None,2014-12-02,6,"5363 windy ridge rd, Charlottesville, VA 22903",Brownsville Elementary,spring 2026


In [37]:
for season in df_peachtree_history_orig["season"].unique():
    # normalize: "spring 2025" → "spring2025"
    var_name = "df_" + season.lower().replace(" ", "")
    
    # create the dataframe
    globals()[var_name] = df_peachtree_history_orig[
        df_peachtree_history_orig["season"] == season
    ].copy()
    
    # print the variable name so you know what was created
    print(f"Created dataframe: {var_name}")

Created dataframe: df_fall2024
Created dataframe: df_fall2025
Created dataframe: df_spring2024
Created dataframe: df_spring2025
Created dataframe: df_spring2026


In [59]:
# IDs in each season
ids_spring25 = set(df_spring2025["peachtree_id"])
ids_fall25   = set(df_fall2025["peachtree_id"])
ids_2026     = set(df_spring2026["peachtree_id"])

# Remove IDs that appear in both 2025 seasons
unique_spring25_ids = ids_spring25 - ids_fall25
unique_fall25_ids   = ids_fall25 - ids_spring25

# Remove anyone who appears in 2026
unique_spring25_ids = unique_spring25_ids - ids_2026
unique_fall25_ids   = unique_fall25_ids - ids_2026

# Filter the dataframes
df_spring2025_unique = df_spring2025[
    df_spring2025["peachtree_id"].isin(unique_spring25_ids)
].copy()

df_fall2025_unique = df_fall2025[
    df_fall2025["peachtree_id"].isin(unique_fall25_ids)
].copy()

In [69]:
df_spring_fall_2025 = (
    pd.concat([df_spring2025_unique, df_fall2025_unique], ignore_index=True)
      .drop_duplicates(subset="peachtree_id")
      .reset_index(drop=True)
)

In [70]:
df_spring_fall_2025 = df_spring_fall_2025[
    df_spring_fall_2025["division"] != "Refunded"
].copy()

In [71]:
df_spring_fall_2025

,peachtree_id,player_name,parent_name,guardian_email,league,division,season_team_name,birthdate,previous_seasons,full_address,school_name,season
0,552,Molly Diehl,Gretchen Diehl,diehlgretchen@gmail.com,Tball,Tball,BT Royals,2019-08-19,0,"1420 Pinedale Road, Charlottesville, VA 22901",First Baptist,spring 2025
1,637,Sage Leblanc,ADAM LEBLANC,ladam87@yahoo.com,Tball,Tball,BT Giants,2020-08-17,0,"9205 MASON DR, DENHAM SPRINGS, LA 70726",unknown,spring 2025
2,621,Roger Baber Iv,Jessica Baber,gapvolley09@gmail.com,Tball,Tball,IT Yankees,2019-12-30,0,"360 Gillums Ridge Rd, Charlottesville, VA 22903",Crozet Avenue Christian,spring 2025
3,131,Caleb Gerlack,Tiffany Patterson,tiffanypatterson1985@gmail.com,Greenwood,Greenwood,None,2010-02-14,4,"6082 Rockfish Gap Tpke, Crozet, VA 22932",WAHS,spring 2025
4,527,Mia Fowler,Harley Gentry,ayyoharley@gmail.com,Softball,10u Softball,None,2014-08-16,1,"3190 Audubon Ct, Earlysville, VA 22936",broadus wood elementary,spring 2025
...,...,...,...,...,...,...,...,...,...,...,...,...
287,349,Italo Coppiano,Stephanie Portillo,stephaniecheer123@gmail.com,Baseball,Rookie Baseball,Rookie Red Sox,2017-02-01,0,"120 Jordan ln, Charlottesville, VA 22903",Brownsville Elementary,fall 2025
288,78,Baylor Smith,Heather Smith,hsmith@cardiac-education.com,Tball,Tball,TB Mets,2020-02-20,0,"5979 cling lane, Crozet, VA 22932",Crozet Elementary,fall 2025
289,669,Sutton Longwell,Elizabeth Longwell,elizabethlongwell1@gmail.com,Tball,Tball,TB Phillies,2020-01-04,0,"2560 Williston Dr, Charlottesville, VA 22901",Ivy Elementary,fall 2025
290,150,Carson Winchell,Kevin Winchell,kwinchell@gmail.com,Tball,Tball,None,2019-09-19,0,"812 Stargazer Lane, Crozet, VA 22932",Crozet Elementary,fall 2025


In [72]:
today = pd.Timestamp.today()
cutoff_date = today - pd.DateOffset(years=18)

df_spring_fall_2025["birthdate"] = pd.to_datetime(df_spring_fall_2025["birthdate"], errors="coerce")

df_spring_fall_2025 = df_spring_fall_2025[df_spring_fall_2025["birthdate"] > cutoff_date].copy()

In [73]:
df_spring_fall_2025.sort_values(by="peachtree_id").reset_index(drop=True)


,peachtree_id,player_name,parent_name,guardian_email,league,division,season_team_name,birthdate,previous_seasons,full_address,school_name,season
0,7,Abraham Williams,Kyle Williams,williams.kyle.edward@gmail.com,Baseball,Rookie Baseball,Rookie Rockies,2016-09-07,2,"228 Florence Ave, Waynesboro, VA 22980",Homeschool,fall 2025
1,8,Adaleigh Birckhead,Candie Loya,cloya6265@gmail.com,Softball,10u Softball,10U Select,2015-12-21,3,"2169 Dudley Mountain Rd, North Garden, VA 22959",red hill elementary,fall 2025
2,11,Addison Pietro,Justin Pietro,jpietro@gmail.com,Softball,10u Softball,"Peachtree 8U All Stars - Dority, 10U Pumas",2016-09-08,1,"2939 Rambling Brook Lane, Crozet, VA 22932",Brownsville Elementary,spring 2025
3,14,Aiden Taylor,Ryan Taylor,schwammi07@yahoo.com,Baseball,Minors Baseball,Minors Mariachis,2014-09-08,4,"3968 Dick Woods Rd, Charlottesville, VA 22903",Covenant,spring 2025
4,15,Ainsley Dority,Mandy Dority,mandydority@icloud.com,Softball,8u Softball,Peachtree 8U All Stars - Dority,2016-10-07,1,"PO BOX 411, Batesville, VA 22924",Brownsville Elementary,spring 2025
...,...,...,...,...,...,...,...,...,...,...,...,...
276,754,Xavier Cauls,Renae Weber,renae1091@gmail.com,Greenwood,Greenwood,None,2011-08-05,10,"775 Newtown Rd, Greenwood, VA 22943",henley middle,spring 2025
277,755,Xavier Fox,Zachary Fox,fox.zachary@gmail.com,Baseball,Rookie Baseball,"Peachtree Rookie All Stars - Downs, Rookie Roc...",2018-01-29,5,"1105 Highlands Drive, Charlottesville, VA 22901",Crozet Elementary,spring 2025
278,758,Zachary Hopkins,Keri Hopkins,kerihopkins83@gmail.com,Tball,Tball,BT Orioles,2020-01-20,0,"2449 Holkham Drive, Charlottesville, VA 22901",C-Street Cooperative,spring 2025
279,760,Zach Hamernick,Cassandra Hamernick,cascarmichael@live.com,Greenwood,Greenwood,None,2009-10-08,8,"2529 Holkham Drive, Charlottesville, VA 22901",WAHS,spring 2025


In [74]:
for league in df_spring_fall_2025["league"].unique():
    # normalize: "spring 2025" → "spring2025"
    var_name = "df_" + league.lower().replace(" ", "")
    
    # create the dataframe
    globals()[var_name] = df_spring_fall_2025[
        df_spring_fall_2025["league"] == league
    ].copy()
    
    # print the variable name so you know what was created
    print(f"Created dataframe: {var_name}")

Created dataframe: df_tball
Created dataframe: df_greenwood
Created dataframe: df_softball
Created dataframe: df_baseball


In [75]:
output_path = "peachtree_players_to_contact_spring2026.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    df_softball.to_excel(writer, sheet_name="softball", index=False)
    df_tball.to_excel(writer, sheet_name="tball", index=False)
    df_baseball.to_excel(writer, sheet_name="baseball", index=False)
    df_greenwood.to_excel(writer, sheet_name="greenwood", index=False)

print("Excel file created:", output_path)

Excel file created: peachtree_players_to_contact_spring2026.xlsx


In [76]:
df_tball

,peachtree_id,player_name,parent_name,guardian_email,league,division,season_team_name,birthdate,previous_seasons,full_address,school_name,season
0,552,Molly Diehl,Gretchen Diehl,diehlgretchen@gmail.com,Tball,Tball,BT Royals,2019-08-19,0,"1420 Pinedale Road, Charlottesville, VA 22901",First Baptist,spring 2025
1,637,Sage Leblanc,ADAM LEBLANC,ladam87@yahoo.com,Tball,Tball,BT Giants,2020-08-17,0,"9205 MASON DR, DENHAM SPRINGS, LA 70726",unknown,spring 2025
2,621,Roger Baber Iv,Jessica Baber,gapvolley09@gmail.com,Tball,Tball,IT Yankees,2019-12-30,0,"360 Gillums Ridge Rd, Charlottesville, VA 22903",Crozet Avenue Christian,spring 2025
5,441,Leila Graham,Christina Graham,cgraham0604@gmail.com,Tball,Tball,BT Orioles,2021-02-24,0,"5630 McKinley Ln, Charlottesville, VA 22903",Daylily,spring 2025
10,674,Taylor Keyton,Erica McFadden,enm29d@gmail.com,Tball,Tball,IT Yankees,2020-09-23,0,"5525 Patrick Henry highway, Roseland, VA 22967",Tye River Elementary School,spring 2025
...,...,...,...,...,...,...,...,...,...,...,...,...
283,329,Henry Sefter,Clay Sefter,csefter@gmail.com,Tball,Tball,TB Mets,2021-08-05,0,"1302 Tilman Rd, Charlottesville, VA 22901",Olivet,fall 2025
286,462,Lily Dass,Lucy Smith,lucysmithdass@gmail.com,Tball,Tball,TB Orioles,2021-06-12,0,"5692 Sugar Hollow Rd, Crozet, VA 22932",Olivet,fall 2025
288,78,Baylor Smith,Heather Smith,hsmith@cardiac-education.com,Tball,Tball,TB Mets,2020-02-20,0,"5979 cling lane, Crozet, VA 22932",Crozet Elementary,fall 2025
289,669,Sutton Longwell,Elizabeth Longwell,elizabethlongwell1@gmail.com,Tball,Tball,TB Phillies,2020-01-04,0,"2560 Williston Dr, Charlottesville, VA 22901",Ivy Elementary,fall 2025
